## Quadtree

**Quadtree** *(или квадродерево)* — дерево, каждая внутренняя вершина которого содержит 4 ребёнка. 

Каждой вершине $v$ квадродерева соответствует некоторый квадрат $a$,
детям вершины $v$ соответствуют четверти квадрата $a$.

<img src="src/SimpleQuadtree.svg">

### Проблемы?

Высота дерева может быть очень большой вне зависимости от количества точек в квадродереве.

<img src="src/QuadtreeWorstCase.svg" />

## Composed Quadtree

Назовём квадрат **интересным**, если соответствующая ему вершина дерева имеет хотя бы два непустых ребёнка или является корнем.

Сжатое квадродерево получается сжатием обычного таким образом, чтобы остались только **интересные** квадраты. 

<img src="src/SimpleToComposed.svg"/>

**Лемма (о высоте сжатого квадродерева) :**

Сжатое квадродерево для $n$ точек имеет $O(n)$ вершин и глубину $O(n)$.

### Локализация

**Локализация** - поиск наименьшего интересного квадрата, где геометрически находится интересующая точка. 

<img src="src/QuadtreeLocalisation.svg" />

### Вставка

1. Локализация
2. Вставка точки в найденный интересный квадрат

    
* Если четверть пустая <br>
    <img src="src/QuadtreeInsert_1.svg" />

* Если четверть не пустая <br>
    <img src="src/QuadtreeInsert_2.svg" />

### Удаление

1. Локализация
2. Удаление точки из найденного интересного квадрата

<img src="src/QuadtreeRemove_1.svg" />

Если после удаления найденный квадрат перестал  быть интересным:

<img src="src/QuadtreeRemove_2.svg" />

### Проблемы?

В наихудшем случае высота дерева зависит линейно от количества точек.

<img src="src/ComposedQuadtreeWorstCase.svg" />

## Skip Quadtree

<img src="src/SQTree.svg">

### Локализация

<img src="src/SQTreeLocalisation.svg" />

### Вставка

<img src="src/SQTreeInsert.svg" />

### Удаление

<img src="src/SQTreeRemove.svg" />

**Лемма (о существовании узла на предыдущем уровне):**

Для любого узла на уровне $Q_i$ существует соответствующий ему узел на уровне $Q_{i-1}$.


**Лемма (о количестве шагов локализации на одном уровне):**

На каждом уровне совершается $O(1)$ шагов локализации для любой точки `x`.

Пусть на уровне $Q_i$ поиск точки `x`, начинающийся с корня, проходит по квадратам $p_0, p_1 \dots p_m$. 

Пусть случайная величина $j$ — количество шагов локализации внутри $Q_i$.

$p_{m - j}$ — последний квадрат из $p_0, p_1 \dots p_m$, являющийся интересным в $Q_{i + 1}$.

Оценим мат.ожидание случайной величины $j$. 

Пусть $T$ — множество непустых четвертинок,  встреченных на пути $p_{m - j + 1} \dots p_m$.

Среди $T$ только одна ($\mathrm{Pr}_1$) или ноль ($\mathrm{Pr}_0$) — непустые на слое $Q_{i + 1}$.

$$\displaystyle
\begin{align}
|T| & = t \\
q   & = 1 - p \\ \\
\mathrm{Pr}_0(j) & = q^t \\
\mathrm{Pr}_1(j) & = t \cdot pq^{t-1} \\ \\
\mathop{\mathbb{E}}j
& = \sum\limits_{j = 1}^{m} j \cdot \left(\mathrm{Pr}_0 (j) + \mathrm{Pr}_1 (j) \right) \\
& = \sum\limits_{j = 1}^{m} j \left(q^t + t \cdot pq^{t-1}\right) \\
& = \sum\limits_{j = 1}^{m} j ((1-t) \cdot q^t + t \cdot q^{t-1}) \\
\end{align}
$$

$$j \leq t$$
т.к. количество четвертей в $T$ — хотя бы $j$.

$$\displaystyle
\begin{align}
\mathop{\mathbb{E}}j
&\leq \sum_{j = 1}^{m} j \left(\left(1-j\right) \cdot q^j + t \cdot q^{j-1}\right) \\
&< \sum_{j = 1}^{\infty} j \left(\left(1-j\right) \cdot q^j + j \cdot q^{j-1}\right) \\
&< \sum_{j = 1}^{\infty} \left(2 j \left(j-1\right) \cdot q^{j-1} - j \left(j-1\right) \cdot q^j\right) \\
&< \sum_{j = 1}^{\infty} 2 j (j-1) \cdot q^{j-1} 
 = 2 \sum_{j = 1}^{\infty} \left(q^{j+1}\right)'' \\
&= 2 \left(\sum_{j = 1}^{\infty} q^{j+1}\right)'' 
 = \left(\frac{2 q^2}{\left(1 - q\right)}\right)'' \\
&= 4p^{-3}
\end{align}
$$

**Лемма (о количестве уровней):** 

Количество уровней — $O(\log n)$.

Пусть $h$ — cлучайная величина, обозначающая количество уровней.

$$p(h = k) = p(h < k + 1) - p (h < k)$$

$$p(h < k) = \left(1 - p^k\right)^n$$

$$\displaystyle
\begin{align}
p(h = k)
& = \left(1 - p^{k + 1}\right)^n - \left(1 - p^k\right)^n \\
& < 1 - \left(1 - p^k\right)^n \\
& < np^k
\end{align}
$$

Оценим мат.ожидание $h$:

$$ \mathop{\mathbb{E}}h
= \sum_{k = 1}^{\infty}k \cdot p(h = k)
= \left[\sum_{k = 1}^{\log_{1/p}(n)}k \cdot p(h = k)\right]
+ \left[\sum_{k = \log_{1/p}(n) + 1}^{\infty}k \cdot p(h = k)\right]
$$

Первая сумма:
$$\displaystyle
\begin{align}
\sum_{k = 1}^{\log_{1/p}(n)}k \cdot p(h = k) 
& < \sum_{k = 1}^{\log_{1/p}(n)}\log_{1/p}(n) \cdot p(h = k) \\
& = \log_{1/p}(n) \cdot \sum\limits_{k = 1}^{\log_{1/p}(n)} p(h = k) \\
& = O(\log n)
\end{align}
$$

Вторая сумма:
$$\displaystyle
\begin{align}
\sum_{\log_{1/p}(n) + 1}^{\infty}k \cdot p(h = k)
& < \sum_{\log_{1/p}(n) + 1}^{\infty}k \cdot np^k  
  < \sum_{\log_{1/p}(n)}^{\infty}k \cdot np^k 
  = n \cdot \sum_{\log_{1/p}(n)}^{\infty}k p^k \\
& = n p^{\log_{1/p} n} \cdot \sum_{k=0}^{\infty} \left(k + \log_{1/p} n\right) \cdot p^k \\
& = n p^{\log_{1/p} n} \cdot \left(\sum_{k=0}^{\infty} k p^k + \log_{1/p} n \sum_{k=0}^{\infty} p^k\right) \\
& = n p^{\log_{1/p} n} \cdot \left(O(1) + \log_{1/p} n \cdot O(1)\right) \\
& = n \cdot n^{-1} \cdot \log_{1/p} n \cdot O(1) \\
& = O(\log n)
\end{align}
$$

### Оценка памяти

Сжатое квадродерево на уровне $Q_k$ занимает $O(p^k n)$ памяти.

$$\sum\limits_{k=0}^{O(\log n)} O\left(p^k n\right) < \sum\limits_{k=0}^{\infty} O\left(p^k n\right) = O(n) $$